In [45]:
import os
import json
import cohere
import re
import logging
from datetime import datetime

from langchain_openai import AzureOpenAIEmbeddings
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType
from azure.search.documents._generated.models import QueryCaptionResult

from openai import AzureOpenAI
import tiktoken

from dotenv import load_dotenv

load_dotenv()

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

class AzureEmbeddings:

    def __init__(self):
        pass

    @staticmethod
    def get_embedding():
        return AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

    @staticmethod
    def generate_embeddings(content: str):
        embeddings = AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

        doc_result = embeddings.embed_documents([content])

        return doc_result[0]

openai_client = AzureOpenAI(
    api_key=os.getenv("AZURE_CHAT_OPENAI_API_KEY"), 
    api_version=os.getenv("AZURE_CHAT_OPENAI_API_VERSION"), 
    azure_endpoint=os.getenv("AZURE_CHAT_OPENAI_ENDPOINT")
    )

embeddings_client = AzureEmbeddings()
store_search_url: str = f'https://{os.getenv('AZURE_COGNITIVE_SEARCH_SERVICE_NAME')}.search.windows.net'
search_client = SearchClient(
            store_search_url, os.getenv("AZURE_COGNITIVE_SEARCH_INDEX_NAME"),
            AzureKeyCredential(os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY"))
        )

co = cohere.Client(api_key=os.getenv('COHERE_API_KEY'))

In [91]:
log_file_name = ""


def ensure_directory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)


def log(message):
    global log_file_name
    ensure_directory("logs")
    with open(log_file_name, "a") as file:
        file.write(f"{datetime.now()}\n{message}\n")

In [36]:
conversation_price = 0
conversation_ct = 0
conversation_pt = 0
conversation_tt = 0


def get_conversation_price(new_completion, model="gpt4o"):
    global conversation_price, conversation_ct, conversation_pt, conversation_tt
    USD_price_in_COP = 4100

    if model == "gpt4o-mini":
        ct_price = 0.0000006
        pt_price = 0.00000015
    if model == "gpt4o":
        ct_price = 0.000015
        pt_price = 0.000005

    ct = new_completion.usage.completion_tokens
    pt = new_completion.usage.prompt_tokens
    tt = new_completion.usage.total_tokens
    usd_total_price = (ct * ct_price) + (pt * pt_price)
    cop_total_price = usd_total_price * USD_price_in_COP
    conversation_ct += ct
    conversation_pt += pt
    conversation_tt += tt
    conversation_usd_total_price = (conversation_ct * ct_price) + (
        conversation_pt * pt_price
    )
    conversation_cop_total_price = conversation_usd_total_price * USD_price_in_COP

    print(
        f"💰 Conversation price: ${conversation_cop_total_price} COP (In: {conversation_pt}, Out: {conversation_ct}, Total token: {conversation_tt})\n"
        f"   This message: ${cop_total_price} COP (In: {pt}, Out: {ct}, Total token: {tt}) "
        f""
    )
    log(
        f"💰 Conversation price: ${conversation_cop_total_price} COP (In: {conversation_pt}, Out: {conversation_ct}, Total token: {conversation_tt})\n "
        f"   This message: ${cop_total_price} COP (In: {pt}, Out: {ct}, Total token: {tt}) "
        f""
    )

    return

In [90]:
### 🔑 Definir system prompt y tools

messages = []
user_input = ""

query_prompt = "Utiliza esta herramienta si necesitas más fuentes para responder al usuario con precisión. Genera dos consultas de búsqueda basadas en lo que el usuario te ha preguntado: una consulta de texto completo utilizando sintaxis Lucene y una consulta semántica. Cada una debe contener la idea completa y fuentes específicas si son necesarias. Si la pregunta del usuario aborda cuándo, cómo o qué información, debes incluir eso en las consultas, ya que afectará los resultados. Evitar hacer búsquedas de coincidencias exactas de palabras, ya que es probables que los textos esten mal transcritos. Utiliza búsquedas por campo, especialmente siempre busca en el campo 'content' y utiliza los campos 'title' y 'author' si son útiles para obtener el fragmento correcto del documento. La consulta semántica debe ser más general y capturar el significado completo de lo que el usuario está preguntando. Si necesitas diferentes fuentes, busca cada una con un llamado a esta tool"

search_legal_info = {
    "type": "function",
    "function": {
        "name": "search_legal_info",
        "description": query_prompt,
        "parameters": {
            "type": "object",
            "strict": "true",
            "properties": {
                "full_text_query": {
                    "type": "string",
                    "description": "La consulta de texto completo utilizando sintaxis Lucene. Debe enfocarse en los campos 'content', 'title' y 'author' para encontrar los fragmentos correctos, evitando depender de coincidencias exactas de palabras debido a posibles errores de transcripción en los documentos. Ejemplo: 'content:causales de agravación artículo 267 AND title:ley 599 del 2000'",
                },
                "semantic_query": {
                    "type": "string",
                    "description": "La consulta semántica que captura el significado completo de lo que el usuario está preguntando. Debe ser más general y no utilizar sintaxis específica de campos o operadores lógicos. Ejemplo: 'causales de agravación de la estafa artículo 267 ley 599 del 2000'",
                },
            },
            "required": ["semantic_query", "full_text_query"],
        },
    },
}
get_next_chunk = {
    "type": "function",
    "function": {
        "name": "get_next_chunk",
        "description": "Use this tool to get when an important sources comes incomplete or the content is cut off. This tool will help you retrieve the following section of that given source.",
        "parameters": {
            "type": "object",
            "properties": {
                "source_id": {
                    "type": "string",
                    "description": "the unique identifier of the chunk that is incomplete. eg:'20240719192458csjscpboletinjurisprudencial20181219pdf_chunk30'",
                },
            },
            "required": ["source_id"],
        },
    },
}

response_system_template = f'Eres Ariel, un asistente para la investigación legal en jurisprudencia colombiana. Sigue estas instrucciones al responder:\n\nConsulta de Fuentes:\n\nUtiliza la herramienta "search_legal_info" para buscar información relevante.\nCrea consultas específicas que incluyan toda la información necesaria.\nSi es necesario, genera múltiples consultas para obtener diferentes tipos de documentos.\nRedacción de Respuestas:\n\nSé detallado y preciso, utilizando exclusivamente la información de las fuentes proporcionadas.\nNo incluyas información que no haya sido extraída directamente de las fuentes.\nCitación de Fuentes:\n\nCita el mayor número de fuentes aplicables.\nUtiliza corchetes para referenciar la fuente con el ID del fragmento sin modificarlo, por ejemplo: [12345_id_de_ejemplo].\nNo combines fuentes; lista cada fuente por separado.\nFormato de Respuesta:\n\nEscribe tus respuestas en formato HTML, sin incluir los tags "```html" al principio o al final.\nUtiliza etiquetas HTML estándar como <p>, <strong>, <em>, etc.\nManejo de Fragmentos Incompletos:\n\nSi un fragmento es necesario pero está incompleto, utiliza "get_next_chunk" con el source_id correspondiente para obtener el siguiente fragmento.\nInterpretación de Categorías:\n\nPrioriza las fuentes según su categoría:\nConstitucional\nLegal\nInfralegal\nJurisprudencia\nDoctrina\nDa mayor importancia a los fragmentos más actuales.\nPrecisión Legal:\n\nSi un extracto cita otra fuente (artículo, ley, etc.), búscala directamente para obtener información de primera mano.\nNo confundas leyes o artículos; si el usuario pregunta por una ley específica, ignora extractos de otras leyes.\nConsultas sin Resultados:\n\nSi las fuentes no ayudan a responder la pregunta, responde: "No encuentro información con esos términos, ¿puedes reformular tu consulta?"'

query_system_template = f"""Eres un experto legal que necesita buscar en una base de conocimiento con cientos de miles de documentos, fuentes confiables y precisas para ayudar a responder las consultas del usuario. He traído algunos extractos de documentos que pueden ser útiles para que identifiques fuentes que puedes incluir en tu término de búsqueda si ves que son aplicables. La base de conocimientos está alojada en Azure AI Search y bebes generar un query en formato JSON para buscar allí. 

Los documentos están guardados en fragmentos con los siguientes campos:
- id: Identificador único del fragmento
- title: Título del documento
- author: Autor del documento
- keywords: Tema legal, puede ser: General, Constitucional, Internacional_Publico, Internacional_Privado, Penal, Financiero, entre otros.
- category: Tipo de documento legal. Las opciones son: Jurisprudencia, Doctrina, Constitucional, Legal, Infralegal y Otros_temas_legales. 
- page: Página de la que fue extraído el fragmento.
- year: Año en el que se publicó el documento.
- content: 500 tokens de contenido, fragmento o extracto del documento. Son solo 500 tokens del documento completo.

Requisitos para generar el términos de búsqueda:
- Debes generar un query que sirva para ejecutar una búsqueda semántica e híbrida.
- En 'search_query' debes generar el query completo
- Si identificas el título del documento o nombre del autor del documento que quieres consultar, lo puedes incluir en 'title_query' o 'author_query' pero de todas formas, debes inlcuir esta búsqueda también en 'search_query'. 
- No es necesario que todas tus búsquedas sean específicas para un documento, puedes hacer búsquedas más generales y sin el campo de 'title_search', a menos que estés seguro que toda la información correcta y completa está en un documento específico.
- El query debe contener la idea semánticamente completa e incluir fuentes, títulos, artículos, leyes específicas si son necesarias. 
- El query no debe ser demasiado largo. Sé conciso.
- Si la pregunta del usuario aborda cuándo, cómo o qué información, debes incluir eso en las consultas, ya que afectará los resultados.
- Si hay varias formas de referise a una fuente, incluye los diferentes nombres de esa fuente en el query.
- Si necesitas buscar diferentes fuentes, crea otro JSON con la búsqueda que necesitas.
- Los fragmentos de referencias no deben desviarte de la búqueda inicial del usaurio.
- Asegúrate de incluir artículos, normas, leyes en el query si identificas que necesitas esas fuentes.
- Usa siempre el número de la ley o el artículo si ves que la información se encuentra allí.
"""

query_few_shots = [
    {"role": "user", "content": "¿Qué dice el artículo 103 del Código Penal?"},
    {
        "role": "assistant",
        "content": '{"text_query":"content:homicidio artículo 103 AND title:"ley 599 del 2000" OR "código penal"","semantic_query":"homicidio artículo 103 ley 599 del 2000 código penal","category_weight":{"Constitucional":6,"Legal":5,"Infralegal":4,"Jurisprudencia":3,"Doctrina":2,"Otros_temas_legales":1},"filters":""}',
    },
    {
        "role": "user",
        "content": "¿Cuáles son las causales genéricas de atenuación punitiva?",
    },
    {
        "role": "assistant",
        "content": '{"text_query":"content:causales genéricas de atenuación punitiva AND title:ley 599 del 2000","semantic_query":"causales genéricas de atenuación punitiva ley 599 del 2000 código penal colombiano","category_weight":{"Constitucional":6,"Legal":5,"Infralegal":4,"Jurisprudencia":3,"Doctrina":2,"Otros_temas_legales":1},"filters":""}',
    },
]

In [38]:
def format_search_results(docs_list):
    print(docs_list)
    documents = []
    try:
        for document in docs_list:
            print(document)
            captions: QueryCaptionResult = document.get("@search.captions", "")
            captions_text = " // ".join([caption.text for caption in captions]) if captions is not None else ""
            doc_formatted = {
                "position" : len(documents) + 1,
                "score": document.get("@search.score", 10),
                "rerank": document.get("@search.reranker_score", 10),
                "captions": captions_text,
                "id": document["id"],
                "title": document["title"],
                "author": document["author"],
                "keywords": document["keywords"],
                "category": document["category"],
                "page": document["page"],
                "year": document["year"],
                "has_copyright": document["has_copyright"],
                "file_path": document["file_path"],
                "external_id": document["external_id"],
                "content": document["content"],
            }
            documents.append(doc_formatted)
        log(
            f">>>>>>>> format_search_results:\n"
            f"{json.dumps(documents, indent=4)}"
            )
    except Exception as e:
        print(e)
    print("Sucesssss")
    return documents

def format_search_results_support(docs_list):
    print(docs_list)
    documents = []
    try:
        for document in docs_list:
            print(document)
            doc_formatted = {
                "content": document.get("content", "none"),
            }
            documents.append(doc_formatted)
        log(
            f">>>>>>>> format_search_results:\n"
            f"{json.dumps(documents, indent=4)}"
            )
    except Exception as e:
        print(e)
    print("Sucesssss")
    return documents

def format_reranked_results(docs_list):
    documents = []
    for document in docs_list:
        doc_formatted = {
            "position" : len(documents) + 1,
            "id": document.document.id,
            "relevance": document.relevance_score,
            "score": document.document.score,
            "rerank": document.document.rerank,
            "captions": document.document.captions,
            "title": document.document.title,
            "author": document.document.author,
            "keywords": document.document.keywords,
            "category": document.document.category,
            "page": document.document.page,
            "year": document.document.year,
            "has_copyright": document.document.has_copyright,
            "file_path": document.document.file_path,
            "external_id": document.document.external_id,
            "content": document.document.content,
        }
        documents.append(doc_formatted)
    log(
        f">>>>>>> format_reranked_results:\n"
        f"{json.dumps(documents, indent=4)}"        
    )
    return documents

def filter_docs(doc_list):
    filtered_docs = []
    contents = [doc["content"] for doc in doc_list]
    
    for doc in doc_list:
        score = doc.get("relevance", 10) 
        rerank = doc.get("relevance", 10) 
        relevance = doc.get("relevance", 1)
        if (
            contents.count(doc["content"]) == 1
            and (relevance > 0.65
            and rerank > 2
            or score > 0.025)
        ):
            filtered_docs.append(doc)
    print(f">> 🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...")
    log(f">> 🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...\n"
    f"{json.dumps(filtered_docs, indent=4)}")
    return filtered_docs

def filter_docs_sources(doc_list):
    filtered_docs = []
    contents = [doc["content"] for doc in doc_list]
    
    for doc in doc_list:
        if (
            contents.count(doc["content"]) == 1
        ):
            filtered_docs.append(doc)
    print(f">> 🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...")
    # log(f">> 🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...\n"
    # f"{json.dumps(filtered_docs, indent=4)}")
    return filtered_docs

def add_doc_to_context(docs_list):
    sources = ""
    counter = 0
    
    for document in docs_list:
        counter += 1
        sources += (
            f"Fuente #{counter}\n"
            f"id: {document["id"]}\n"
            f"title: {document["title"]}\n"
            f"author: {document["author"]}\n"
            f"year: {document["year"]}\n"
            f"keywords: {document["keywords"]}\n"
            f"category: {document["category"]}\n"
            f"page: {document["page"]}\n"
            f"content: {document["content"]}\n"
            f"\n\n"
        )      
    log(
        f"To add in Context: \n"
        f"{json.dumps({"sources":sources}, indent=4)}"
        )
    return sources
    

In [73]:
def search_for_chunks(text_query, vector_query, rerank_query):
    log(
        f">> 🔎 Buscando documentos Text Query: {text_query} ...\n"
        f"      Buscando documentos Semantic Query: {rerank_query} ...\n"
    )
    print(
        f">> 🔎 Buscando documentos Text Query: {text_query} ...\n"
        f">>    Buscando documentos Semantic Query: {rerank_query} ...\n"
    )
    results_num = 50
    results = search_client.search(
        search_text=text_query,
        vector_queries=[
            {
                "vector": vector_query,
                "k": results_num,
                "fields": "content_vector",
                "kind": "vector",
                "exhaustive": True,
            }
        ],
        top=results_num,
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="default",
        query_caption="extractive|highlight-false",
        scoring_profile="legal",
        scoring_parameters=[
            "tagx6-Constitucional",
            "tagx5-Legal",
            "tagx4-Infralegal",
            "tagx3-Jurisprudencia",
            "tagx2-Doctrina",
        ],
    )

    # Convertir en Lucene full text syntax y pedir otro query semántico
    #

    results_formatted = format_search_results(results)
    restults_filtered = filter_docs(results_formatted)

    log(f">> ⭐️ Aplicando reorganización semántica: {rerank_query} ...")
    print(f">> ⭐️ Aplicando reorganización semántica ...")
    reranked_docs = co.rerank(
        model="rerank-multilingual-v3.0",
        top_n=10,
        query=rerank_query,
        documents=restults_filtered,
        return_documents=True,
        rank_fields=["content", "title", "author", "keywords", "category"],
    )
    reranked_docs_formatted = format_reranked_results(reranked_docs.results)
    reranked_docs_filtered = filter_docs(reranked_docs_formatted)

    # reranked_docs_filtered = restults_filtered[:10]

    return reranked_docs_filtered


def search_semantic(query):
    # log(
    #     f">> 🔎 Buscando documentos Text Query: {text_query} ...\n"
    #     f"      Buscando documentos Semantic Query: {rerank_query} ...\n"
    # )
    print(f">> 🔎 Buscando documentos de soporte: {query} ...\n")

    vector_query = embeddings_client.generate_embeddings(content=query)

    results_num = 20
    results = search_client.search(
        filter="category eq 'Jurisprudencia' or category eq 'Doctrina'",
        vector_queries=[
            {
                "vector": vector_query,
                "k": results_num,
                "fields": "content_vector",
                "kind": "vector",
                "exhaustive": True,
            }
        ],
        top=results_num,
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="default",
        query_caption="extractive|highlight-false",
    )
    results_formatted = format_search_results_support(results)
    restults_filtered = filter_docs_sources(results_formatted)

    return restults_filtered[:3]


def get_next_chunks(id):
    log(f">> ⏩️ Buscando los chunks siguientes de {id} ...")
    print(f">> ⏩️ Buscando los chunks siguientes de {id} ...")
    pattern = r"^(.*_chunk)(\d+)$"
    match = re.search(pattern, id)

    if match:
        prefix = match.group(1)
        chunk_number = int(match.group(2))
        next_chunks_result = search_client.search(
            filter=f"id eq '{prefix}{chunk_number + 1}' or id eq '{prefix}{chunk_number + 2}'",
            top=2,
        )
        return format_search_results(list(next_chunks_result))


def get_next_chunk_tool(source_id):
    next_chunks = get_next_chunks(source_id)
    log(f"Continuación de {source_id}:\n{add_doc_to_context(next_chunks)}")
    return f"Continuación de {source_id}:\n{add_doc_to_context(next_chunks)}"


def search_legal_info_tool(text_query, semantic_query):

    vector_query = embeddings_client.generate_embeddings(content=semantic_query)

    results = search_for_chunks(
        text_query=text_query,
        rerank_query=semantic_query,
        vector_query=vector_query,
    )
    return f"Extractos de documentos encontrados: \n{add_doc_to_context(results)}"

In [86]:
def generate_completion():
    global messages
    log(f"{json.dumps(messages, indent=4)}\n")
    response = openai_client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        messages=messages,
        tools=[get_next_chunk, search_legal_info],
        temperature=0.2,
        n=1,
        tool_choice="auto",
    )
    log(response)
    get_conversation_price(response)
    return response


def generate_mini_completion():
    global messages
    log(f"{json.dumps(messages, indent=4)}\n")
    response = openai_client.chat.completions.create(
        model=os.getenv("AZURE_CHAT_MINI_OPENAI_DEPLOYMENT"),
        messages=messages,
        # tools=[get_next_chunk, search_legal_info],
        temperature=0.2,
        n=1,
        # tool_choice="auto",
    )
    log(response)
    get_conversation_price(response, model="gpt4o-mini")
    return response


schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "search_query",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "searches": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "search_query": {
                                "type": "string",
                                "description": "El término de búsqueda completo.",
                            },
                            "title_query": {
                                "type": "string",
                                "description": "Campo opcional, solo si estás seguro de cuál es el título o nombre del documento y lo ves necesario para obtener la información para responder al usuario. Si identificas cuál es el nombre del documento que quieres buscar, inclúyelo aquí. Si vas a buscar una ley o un artículo, siempre incluye el número.Este nombre también debe estar incluido en search_query.",
                            },
                            "author_query": {
                                "type": "string",
                                "description": "Campo opcional. Si identificas cuál es el nombre del autor del documento que quieres buscar, inclúyelo aquí. Este nombre también debe estar incluido en search_query.",
                            },
                            "category_weight": {
                                "type": "object",
                                "properties": {
                                    "Constitucional": {
                                        "type": "integer",
                                        "description": "Weight for the Constitucional category. Defaults to 6 in most cases unless specified by the user question.",
                                    },
                                    "Legal": {
                                        "type": "integer",
                                        "description": "Weight for the Legal category. Defaults to 5 in most cases unless specified by the user question.",
                                    },
                                    "Infralegal": {
                                        "type": "integer",
                                        "description": "Weight for the Infralegal category. Defaults to 4 in most cases unless specified by the user question.",
                                    },
                                    "Jurisprudencia": {
                                        "type": "integer",
                                        "description": "Weight for the Jurisprudencia category. Defaults to 3 in most cases unless specified by the user question.",
                                    },
                                    "Doctrina": {
                                        "type": "integer",
                                        "description": "Weight for the Doctrina category. Defaults to 2 in most cases unless specified by the user question.",
                                    },
                                    "Otros_temas_legales": {
                                        "type": "integer",
                                        "description": "Weight for other legal topics. Defaults to 1 in most cases unless specified by the user question.",
                                    },
                                },
                                "required": [
                                    "Constitucional",
                                    "Legal",
                                    "Infralegal",
                                    "Jurisprudencia",
                                    "Doctrina",
                                    "Otros_temas_legales",
                                ],
                                "additionalProperties": False,
                                "description": "El peso en importancia de las categorías. En la gran mayoría de casos, los pesos los distribuirás de la siguiente manera 'category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2'.",
                            },
                            "filters": {
                                "type": "string",
                                "description": "Este campo es opcional y solo si lo ves estrictamente necesario. Este campo va a afectar mucho tu búsqueda, así que solo úsalo si estás 100% convencido de que lo necesitas. Aquí podrás reducir tu búsqueda manipulando los filtros del query utilizando la sintaxis $filter OData de Azure AI Search, ten en cuenta los campos y los valores utilizados para guardar los fragmentos a la hora de generar un filtro.",
                            },
                        },
                        "required": [
                            "search_query",
                            "title_query",
                            "author_query",
                            "category_weight",
                            "filters",
                        ],
                        "additionalProperties": False,
                    },
                }
            },
            "required": ["searches"],
            "additionalProperties": False,
        },
    },
}


def generate_queries():
    global messages
    # log(f"{json.dumps(messages, indent=4)}\n")
    response = openai_client.chat.completions.create(
        model=os.getenv("AZURE_CHAT_OPENAI_DEPLOYMENT"),
        messages=messages,
        response_format=schema,
        temperature=0.0,
    )
    log(response)
    get_conversation_price(response)
    return response


def generate_mini_query():
    global messages
    # log(f"{json.dumps(messages, indent=4)}\n")
    response = openai_client.chat.completions.create(
        model=os.getenv("AZURE_CHAT_MINI_OPENAI_DEPLOYMENT"),
        messages=messages,
        response_format=schema,
        temperature=0.0,
    )
    log(response)
    get_conversation_price(response, model="gpt4o-mini")
    return response

In [41]:
def append_message(new_message):
    global messages
    conversation = messages

    if isinstance(new_message, dict):
        role = new_message.get("role")
        content = new_message.get("content")
    else:
        role = new_message.role
        content = new_message.content

    if role == "assistant":
        if new_message.tool_calls:
            tool_calls = new_message.tool_calls
            tool_calls_formatted = []
            for tool_call in tool_calls:
                tool_calls_formatted.append(
                    {
                        "id": tool_call.id,
                        "function": {
                            "arguments": str(json.loads(tool_call.function.arguments)),
                            "name": tool_call.function.name,
                        },
                        "type": "function",
                    }
                )
            conversation.append(
                {
                    "role": "assistant",
                    "tool_calls": tool_calls_formatted,
                },
            )
            return conversation  ## Assistant call tools
        elif content:
            conversation.append({"role": "assistant", "content": content})
            return conversation  ## Assistant talk
    elif role == "tool":
        conversation.append(new_message)
        return conversation  ## Tool reponse
    else:
        return conversation

In [42]:
def call_tools(tool_calls):
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)
        if tool_name == "get_next_chunk":
            content = get_next_chunk_tool(tool_args.get("source_id"))
        elif tool_name == "search_legal_info":
            content = search_legal_info_tool(
                text_query=tool_args.get("full_text_query"),
                semantic_query=tool_args.get("semantic_query"),
            )

        new_tool_message = {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": tool_name,
            "content": content,
        }

        append_message(new_message=new_tool_message)

    print(">> 🤖 Generando respuesta ...")
    new_assistant_completion = generate_completion()
    new_assistant_message = new_assistant_completion.choices[0].message
    append_message(new_message=new_assistant_message)

    if new_assistant_message.content:
        return print("💬 Assistant:" + new_assistant_message.content)
    else:
        return call_tools(
            tool_calls=new_assistant_message.tool_calls,
        )

In [43]:
def run_conversation(user_prompt=None):
    if user_prompt:
        get_answer(user_prompt)
    else:
        while True:
            user_input = input("You: ")
            if user_input.lower() == "exit":
                print("Exiting chat...")
                break
            get_answer(user_input)


def get_answer(user_prompt):
    global messages, log_file_name, user_input

    user_input = user_prompt

    current_time = datetime.now().strftime("%m-%d %H:%M")
    log_file_name = f"logs/{current_time} - {user_input[:50]}.log"

    print("🙎‍♂️ User: " + user_input)
    log(f"User input: {user_input}")
    new_user_message = {"role": "user", "content": user_input}

    messages = [
        system_message,
        new_user_message,
    ]

    print(">> 🤖 Generando respuesta ...")
    response = generate_completion()
    response_message = response.choices[0].message

    messages = append_message(new_message=response_message)

    tool_calls = response_message.tool_calls
    if tool_calls:
        call_tools(tool_calls=tool_calls)
    elif response_message.content:
        print("💬 Assistant:", response_message.content)
    else:
        print("An error ocurred during completion generation - response: ", response)

In [28]:
log_file_name = "logs/0 - Artículo 103 del codigo penal.log"

In [78]:
messages = []
user_input = "¿Cuáles son las causales de agravación de la estafa?"

In [63]:
query_prompt_message = {"role": "system", "content": query_system_template}
user_prompt_message = {
    "role": "user",
    "content": f"Genera uno o varias búsquedas para esta pregunta: {user_input}",
}

messages = [
    query_prompt_message,
    # *query_few_shots,
    user_prompt_message,
]
print(json.dumps(messages, indent=4))

[
    {
        "role": "system",
        "content": "Eres un experto legal que necesita buscar en una base de conocimiento con cientos de miles de documentos, fuentes confiables y precisas para ayudar a responder las consultas del usuario. He tra\u00eddo algunos extractos de documentos que pueden ser \u00fatiles para que identifiques fuentes que puedes incluir en tu t\u00e9rmino de b\u00fasqueda si ves que son aplicables. La base de conocimientos est\u00e1 alojada en Azure AI Search y bebes generar un query en formato JSON para buscar all\u00ed. \n\nLos documentos est\u00e1n guardados en fragmentos con los siguientes campos:\n- id: Identificador \u00fanico del fragmento\n- title: T\u00edtulo del documento\n- author: Autor del documento\n- keywords: Tema legal, puede ser: General, Constitucional, Internacional_Publico, Internacional_Privado, Penal, Financiero, entre otros.\n- category: Tipo de documento legal. Las opciones son: Jurisprudencia, Doctrina, Constitucional, Legal, Infralega

In [64]:
response = generate_mini_query()

💰 Conversation price: $8.951324999999999 COP (In: 12799, Out: 439, Total token: 13238)
   This message: $0.529515 COP (In: 573, Out: 72, Total token: 645) 


In [32]:
print(response)

ChatCompletion(id='chatcmpl-A7NrSKmwUHILdPIMFQdq5cBaQvzPZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"searches":[{"search_query":"causales de agravación de la estafa","title_query":"causales de agravación de la estafa","author_query":"","category_weight":{"Constitucional":6,"Legal":5,"Infralegal":4,"Jurisprudencia":3,"Doctrina":2,"Otros_temas_legales":1},"filters":""}]}', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1726323382, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_b2ffeb16ee', usage=CompletionUsage(completion_tokens=82, prompt_tokens=545, total_tokens=627), prompt_filter_results=[{'prompt_index': 0, 'content_filter_resul

In [65]:
log_file_name = "logs/1 - Queries.log"
search_text_query = f"causales de agravación de la estafa"
# Add the category weight here
log(
    f"\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n\n>>>>>>>>>>>>>>>>>>>>>>>>>> 🔎🔎 Text: {search_text_query}\n>>>>>>>>>>>>>>>>>>>>>>>>>> 🔎🔎 Semantic: {search_text_query}\n\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n>>>>>>>>>>>>>>>>>>>>>>>>>>"
)
content = search_legal_info_tool(
    text_query=search_text_query, semantic_query=search_text_query
)

>> 🔎 Buscando documentos Text Query: causales de agravación de la estafa ...
>>    Buscando documentos Semantic Query: causales de agravación de la estafa ...

<iterator object azure.core.paging.ItemPaged at 0x158bdf170>


KeyboardInterrupt: 

In [66]:
print(content)

Extractos de documentos encontrados: 
Fuente #1
id: 20240825050857csjsentencia2660114032007pdf_chunk5
title: CSJ - Sentencia 26601(14-03-2007)
author: Corte Suprema de Justicia
year: 2007
keywords: Penal
category: Jurisprudencia
page: 7
content: actualmente —Ley 599 de 2000- en los artículos 411 —Tráfico de
Influencias-, y 247 (én concordancia con el art. 246 ibidem), que
regula el delito de Estafa con circunstancias de agravación
punitiva, para concluir,. conforme lo atribuido al encartado, que la
ilicitud mutó en su denominación típica, a esta última figura,
ahora asimilada a un punible contra el patrimonio económico, de
%r'¿¿*& de %>4ú¡má"¿&
| Página 8 de 40
] “S e Segunda instancia N* 26.601
S BELISARIO MORENO RE
%nr¿e gyórm¿a» Je%á!¿m'c¿
conformidad con lo que sobre el particular ha dejado sentado la
Sala Penal de la H. Corte Suprema de Justicia.
Concluye, así, la sentencia impugnada, que la conducta
punible siempre ha estado delimitada como tal, aunque, con la
expedición de la Le

In [49]:
codes_to_laws = "\n- Código Penal: Ley 599 de 2000\n- Código Civil: Ley 57 de 1887\n- Código de Comercio: Decreto 410 de 1971\n- Código de Procedimiento Civil: Decreto 1400 de 1970\n- Código de Procedimiento Penal: Ley 906 de 2004\n- Código General del Proceso: Ley 1564 de 2012\n- Código de la Infancia y la Adolescencia: Ley 1098 de 2006\n- Código Nacional de Policía: Decreto 1355 de 1970\n- Código de Recursos Naturales: Decreto 2811 de 1974\n- Código Electoral: Decreto 2241 de 1986\n- Código Disciplinario Único: Ley 734 de 2002\n- Código Contencioso Administrativo: Ley 1437 de 2011 (anteriormente Decreto 1 de 1984)\n- Código de Minas: Ley 685 de 2001\n- Código de Educación: Ley 115 de 1994\n- Código Nacional de Tránsito Terrestre: Ley 769 de 2002\n- Código del Menor: Decreto 2737 de 1989\n- Código de Construcción del Distrito Capital de Bogotá: Acuerdo 20 de 1995\n- Código de Construcción Sismo-Resistente: Acuerdo correspondiente (no especificado en los resultados)\n- Código de Régimen Departamental: Decreto 1222 de 1986\n- Código de Régimen Político y Municipal: Decreto-Ley 1333 de 1986\n- Código Penal Militar: Ley 522 de 1999\n- Código Penitenciario y Carcelario: Ley 65 de 1993\n- Código Procesal del Trabajo y del Seguro Social: Decreto-Ley 2158 de 1948\n- Código Sustantivo del Trabajo: Decreto 2663 de 1950\n- Código Sanitario Nacional: Ley 9 de 1979\n- Código General Disciplinario: Ley 1952 de 2019\n- Código Nacional de Seguridad y Convivencia Ciudadana: Ley 1801 de 2016\n- CODIGO DE POLICIA DE BOGOTA: ACUERDO 79 DE 2003\n- Código Penal Militar: Ley 522 de 1999\n- Código Penitenciario y Carcelario: Ley 65 de 1993\n- Código Procesal del Trabajo y del Seguro Social: Decreto-Ley 2158 de 1948\n- Código Sustantivo del Trabajo: Decreto 2663 de 1950\n- Código Sanitario Nacional: Ley 9 de 1979\n- Código General Disciplinario: Ley 1952 de 2019\n- Código Nacional de Seguridad y Convivencia Ciudadana: Ley 1801 de 2016"

In [67]:
query_prompt_message = {"role": "system", "content": query_system_template}
user_prompt_message = {
    "role": "user",
    "content": f"Genera uno o varias búsquedas para esta pregunta: {user_input}",
}
sources_instructions_prompt_message = {
    "role": "system",
    "content": f"Ejectuaste una búsqueda inicial con el query 'causales de agravación de la estafa' y encontraste algunos extractos con esta búsqueda. Ahora, sigue las siguientes instrucciones:\n1. Revisa todos los extractos obtenidos con tu primer query e identifica fuentes relevantes que te ayuden a responder al usuario.\n2. Identifica leyes, artículos, o nombres de sentencias o documentos exactos e inclúyelos en el query (Estos también deben ser buscados como títulos).\n3. Si la ley, artículo o sentencia tiene un número ej: 'Ley 203 de 1999', inclúyela en el query junto a su nombre de referencia 'Código de ...'.\n4. Arma tu query teniendo en cuenta las fuentes, incluyendo el contexto semántico completo, pero sé conciso y no busques varias cosas en un solo query.\n5. Si necesitas buscar más de una fuente o término, crea varios objetos de query.\n\nEsta es una lista de los códigos en colombia y sus respectiva ley, debes incluir ambas refencias en tu query: {codes_to_laws}",
}
sources_prompt_message = {
    "role": "assistant",
    "content": f"Resultados obtenidos al buscar 'causales de agravación de la estafa': \n {content} \n\n Ahora vuelve a generar uno o más queries pero mejor que el anterior, más preciso y sin desviarte de la pregunta inicial del usuario: '{user_input}'",
}

messages = [
    query_prompt_message,
    # *query_few_shots,
    user_prompt_message,
    sources_instructions_prompt_message,
    sources_prompt_message,
]

print(json.dumps(messages, indent=4))

[
    {
        "role": "system",
        "content": "Eres un experto legal que necesita buscar en una base de conocimiento con cientos de miles de documentos, fuentes confiables y precisas para ayudar a responder las consultas del usuario. He tra\u00eddo algunos extractos de documentos que pueden ser \u00fatiles para que identifiques fuentes que puedes incluir en tu t\u00e9rmino de b\u00fasqueda si ves que son aplicables. La base de conocimientos est\u00e1 alojada en Azure AI Search y bebes generar un query en formato JSON para buscar all\u00ed. \n\nLos documentos est\u00e1n guardados en fragmentos con los siguientes campos:\n- id: Identificador \u00fanico del fragmento\n- title: T\u00edtulo del documento\n- author: Autor del documento\n- keywords: Tema legal, puede ser: General, Constitucional, Internacional_Publico, Internacional_Privado, Penal, Financiero, entre otros.\n- category: Tipo de documento legal. Las opciones son: Jurisprudencia, Doctrina, Constitucional, Legal, Infralega

In [68]:
response = generate_mini_query()

💰 Conversation price: $13.002945 COP (In: 18619, Out: 631, Total token: 19250)
   This message: $4.051620000000001 COP (In: 5820, Out: 192, Total token: 6012) 


In [69]:
print(response)
print("\n\n")
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-A7PCWhmcLIZLrh4Y8JOzv0eiSNUju', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"searches":[{"search_query":"causales de agravación de la estafa Código Penal Ley 599 de 2000 artículo 247","title_query":"Ley 599 de 2000","author_query":"Congreso","category_weight":{"Constitucional":6,"Legal":5,"Infralegal":4,"Jurisprudencia":3,"Doctrina":2,"Otros_temas_legales":1},"filters":""},{"search_query":"CSJ Sentencia 26601 14-03-2007 agravación de la estafa","title_query":"CSJ - Sentencia 26601(14-03-2007)","author_query":"Corte Suprema de Justicia","category_weight":{"Constitucional":6,"Legal":5,"Infralegal":4,"Jurisprudencia":3,"Doctrina":2,"Otros_temas_legales":1},"filters":""}]}', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 

ChatCompletion(id='chatcmpl-A7OuAcln4x7AWVnfyNR0dSbetiEFc', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"searches":[{"search_query":"causales de agravación de la estafa artículo 247 Código Penal Ley 599 de 2000","title_query":"artículo 247 Código Penal Ley 599 de 2000","author_query":"","category_weight":{"Constitucional":6,"Legal":5,"Infralegal":4,"Jurisprudencia":3,"Doctrina":2,"Otros_temas_legales":1},"filters":""},{"search_query":"Ley 1474 de 2011 causales de agravación de la estafa","title_query":"Ley 1474 de 2011","author_query":"","category_weight":{"Constitucional":6,"Legal":5,"Infralegal":4,"Jurisprudencia":3,"Doctrina":2,"Otros_temas_legales":1},"filters":""}]}', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1726327394, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_b2ffeb16ee', usage=CompletionUsage(completion_tokens=178, prompt_tokens=6425, total_tokens=6603), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])

ChatCompletion(id='chatcmpl-A7P5BpFIwJXTOkyiM8MilzOyzBsTI', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"searches":[{"search_query":"causales de agravación de la estafa Código Penal Ley 599 de 2000 artículo 247","title_query":"Ley 599 de 2000","author_query":"Congreso","category_weight":{"Constitucional":6,"Legal":5,"Infralegal":4,"Jurisprudencia":3,"Doctrina":2,"Otros_temas_legales":1},"filters":""},{"search_query":"CSJ Sentencia 26601 14-03-2007 agravación de la estafa","title_query":"Sentencia 26601(14-03-2007)","author_query":"Corte Suprema de Justicia","category_weight":{"Constitucional":6,"Legal":5,"Infralegal":4,"Jurisprudencia":3,"Doctrina":2,"Otros_temas_legales":1},"filters":""}]}', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1726328077, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_b2ffeb16ee', usage=CompletionUsage(completion_tokens=189, prompt_tokens=5801, total_tokens=5990), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])


In [ ]:
print(response)
print("\n\n")
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-A79c5CtwIGT9CPhr7XemCEdoEFV5Z', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"search_query":"causales de agravación de la estafa artículo 247 Código Penal","title_query":"artículo 247 Código Penal","author_query":"","category_weight":{"Constitucional":6,"Legal":5,"Infralegal":4,"Jurisprudencia":3,"Doctrina":2,"Otros_temas_legales":1},"filters":""}', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1726268613, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_b2ffeb16ee', usage=CompletionUsage(completion_tokens=78, prompt_tokens=1886, total_tokens=1964), prompt_filter_results=[{'prompt_index': 0, 'content_filter_re

In [74]:
log_file_name = "logs/1 - Queries.log"
# category_weight = (
#     "AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2"
# )
search_semantic_query = (
    "causales de agravación de la estafa artículo 247 Código Penal Ley 599 de 2000"
)
search_text_query = (
    f"causales de agravación de la estafa artículo 247 Código Penal Ley 599 de 2000"
)
# Add the category weight here
log(
    f"\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n\n>>>>>>>>>>>>>>>>>>>>>>>>>> 🔎🔎 Text: {search_text_query}\n>>>>>>>>>>>>>>>>>>>>>>>>>> 🔎🔎 Semantic: {search_semantic_query}\n\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n>>>>>>>>>>>>>>>>>>>>>>>>>>"
)
content = search_legal_info_tool(
    text_query=search_text_query, semantic_query=search_semantic_query
)

>> 🔎 Buscando documentos Text Query: causales de agravación de la estafa artículo 247 Código Penal Ley 599 de 2000 ...
>>    Buscando documentos Semantic Query: causales de agravación de la estafa artículo 247 Código Penal Ley 599 de 2000 ...

<iterator object azure.core.paging.ItemPaged at 0x126d64da0>
{'content': "de estafa agravada por razón de la cuantía, el cual tenfa en el\nCódigo Penal de 1.980 una sanción punitiva de uno (1) a diez (10)\na-ños, incrementada de una tercra parte a la mitad acorde con el\nartículo 372, en la Léy 599 de 2.000 se eliminó la agravación\npunitiva para el punible de estafa tipificado en el artículo 246, toda\nvez que no se encuentra contemplada por el artículo 247.\n3. Siendo ello así y en el entendido de que la prescripción de la\nacción penal corresponde a un tiempo igual al máximo de la pena\nNN\nRepública de Colombia | Casación No. 23413\nP./ Carlos Emilio Posada Gonzátez\nCorte Suprema de Justicia\nfijada en la ley, que lo seríá de ocho (8) años, 

In [72]:
# Probar con all > Dejarlo default mejor
# Probar full text search > Malos resultados
# Filtrar por semantic rerank score de 2 o más.
# Si no hay filtro por title, o author, solo searchField content
# Otro proceso del mini 4o donde seleccione las mejores fuentes para responder la respuesta

In [76]:
good_sources = "Fuente #1\nid: 20240824144301csjsp2341315062005pdf_chunk1\ntitle: CSJ - SP23413(15-06-2005)\nauthor: Corte Suprema de Justicia\nyear: 2005\nkeywords: Penal\ncategory: Jurisprudencia\npage: 2\ncontent: de estafa agravada por raz\u00f3n de la cuant\u00eda, el cual tenfa en el\nC\u00f3digo Penal de 1.980 una sanci\u00f3n punitiva de uno (1) a diez (10)\na-\u00f1os, incrementada de una tercra parte a la mitad acorde con el\nart\u00edculo 372, en la L\u00e9y 599 de 2.000 se elimin\u00f3 la agravaci\u00f3n\npunitiva para el punible de estafa tipificado en el art\u00edculo 246, toda\nvez que no se encuentra contemplada por el art\u00edculo 247.\n3. Siendo ello as\u00ed y en el entendido de que la prescripci\u00f3n de la\nacci\u00f3n penal corresponde a un tiempo igual al m\u00e1ximo de la pena\nNN\nRep\u00fablica de Colombia | Casaci\u00f3n No. 23413\nP./ Carlos Emilio Posada Gonz\u00e1tez\nCorte Suprema de Justicia\nfijada en la ley, que lo ser\u00ed\u00e1 de ocho (8) a\u00f1os, pero que este 'se\nreducea la mitad una vez proferida la resoluci\u00f3n acusatoria, sin que\nsea menor a cinco (5) a\u00f1os, precisamente \u00e9ste es el lapso\ncorrespondiente en que la acci\u00f3n penal prescribir\u00eda en este caso,\nt\u00e9rmino que se habr\u00eda cumplido como quiera que la acusaci\u00f3n\n\u201cqued\u00f3 en firme\u201d el 13 de marzo de 2.000.\nSolicita, as\u00ed, se decrete la prescripci\u00f3n de la acci\u00f3n penal a favor del\nprocesado.\n4. Ciertamente, como lo hace notar el memorialista, el art\u00edculo 247\ndel tC\u00f3\u00e9digo Penal actualmente vigent\u00e9 contempl\u00f3 como\ncircunstancias espec\u00edficas agravatorias para el delito contra el\npatrimonio econ\u00f3mico.de estafa: - que el medio fraudulento utilizado\ntenga relaci\u00f3n con vivienda de inter\u00e9\u00e9 social; - que el provecho il\u00edcito\nse obtenga por quien sin ser part\u00edcipe de un delito de secue\u00e9tro o\nextorsi\u00f3n, con ocasi\u00f3n del mismo, induzca o mantenga a otra en\nerror; y \u2014 cuando se invoquen influencias reales o simuladas con el |\n\n\nFuente #2\nid: 20240823230318csjsp1728319032002pdf_chunk18\ntitle: CSJ - SP17283(19-03-2002)\nauthor: Corte Suprema de Justicia\nyear: 2002\nkeywords: Penal\ncategory: Jurisprudencia\npage: 18\ncontent: provecho il\u00edcito personal cor: el correlativo desmedro al prestigio de la\nadministraci\u00f3n de -justicia y al patrimonio de la v\u00edctima; pues la\nacriminaci\u00f3n legal de esta conducta no deja duda de que el punible es\npluriofensivo, en trat\u00e1ndose de la circunstancia de agravaci\u00f3n espec\u00edfica\nprevista para la estafa en el numeral 3* del art\u00edculo 247 C\u00f3digo Penal\nvigente.\nAA\nREPUBLICA DE COLOMBIA 19 CASACI\u00d3N No. 17.283\nSANTIAGO SALAH ARGUELLO\nCorte Suprema de Justicia\n6-. La exposici\u00f3n de motivos del nuevo C\u00f3digo Penal (Ley 599 de\n2000), contenida en el oficio del 4 de agosto de 1998, que el Fiscal\nGeneral de la Naci\u00f3n envi\u00f3 al Presidente del Senado de la Rep\u00fablica,\ncuyo texto fue pubiicado por la Fiscal\u00eda, corrobora que el tr\u00e1fico de\ninfluencias de particular definido en el art\u00edculo 147 del anterior C\u00f3digo\nPenal, corresponde en la nueva normatividad a una circunstancia de\nagravaci\u00f3n punitiva de la estafa: '\n\"Para el delito de estafa se establecen nuevas circunstancias de\nagravaci\u00f3n: la primera referida a cuando el medio fraudulento tenga relaci\u00f3n\ncon vivienda de inter\u00e9s social, como forma de reacci\u00f3n contra una\ncostumbre que expiota la necesidad de los sectores menos favorecidos; la\nsegunda, que hac\u00eda parte de la ley 40 de 1.993, relativa al provecho il\u00edcito\nderivado de un delito de secuestro o extorsi\u00f3n; la tercera corresponde al\nvigente art\u00edculo 147 denominado Tr\u00e1fico de influencias para obtener favor\nde servidor p\u00fablico.\u201d\n7-. Tanto la sentencia del 12 de mayo de 1999 proferida por el\nJuzgado Veintitr\u00e9s Penal del Circuito, como la de segunda instancia\nconfirmatoria de la _ante\u00f1or, dictada el 20 de agosto de 1999 por el\n\n\nFuente #3\nid: 20240827165051jepautotpsa119103agostode2022pdf_chunk2\ntitle: JEP - Auto TP SA 1191 03 agosto de 2022\nauthor: Justicia Especial para la Paz\nyear: 2022\nkeywords: Transicional\ncategory: Jurisprudencia\npage: 2\ncontent: del 9 de mayo de 2018.\n2. Seg\u00fan el fallo condenatorio, los hechos que motivaron la condena sucedieron en la\nvereda Suesc\u00fan del municipio de Tibasosa, Boyac\u00e1, el 12 de diciembre de 2007, cuando dos\nindividuos incineraron con gasolina un bus de servicio p\u00fablico de propiedad del se\u00f1or\nRogelio SILVA SILVA, afiliado a la empresa Los Libertadores, en momentos en que cubr\u00eda\nla ruta Bogot\u00e1 \u2013 Sogamoso. Del proceso penal se extrae que: (i) el citado veh\u00edculo estaba\namparado por una p\u00f3liza gubernamental contra actos terroristas y otra empresarial para\ncubrir indemnizaciones por percances de todo tipo, incluidos actos violentos; (ii) el se\u00f1or\nSILVA SILVA plane\u00f3 la incineraci\u00f3n del veh\u00edculo con la finalidad de hacer efectivo el\npago de las p\u00f3lizas de seguro; de esta forma esperaba obtener un veh\u00edculo nuevo y\nsuperar una dif\u00edcil situaci\u00f3n econ\u00f3mica; (iii) para ello, procur\u00f3 que fuera la guerrilla que\noperaba en Arauca la que ocasionara el siniestro6, pero ante el fracaso de esta iniciativa7\n2 Folio 151-195. Asimismo, el Juzgado declar\u00f3 la prescripci\u00f3n de la acci\u00f3n penal correspondiente al delito de estafa\nagravada, atribuida en la acusaci\u00f3n en el grado de tentativa conforme el art\u00edculo 246 del C\u00f3digo Penal y numeral 4 del\nart\u00edculo 247 de la misma obra.\n3 Art\u00edculo 103 del C\u00f3digo Penal, Ley 599 de 2000, con las causales de agravaci\u00f3n consagradas en el art\u00edculo 104, numerales\n2, 4 y 7 del mismo estatuto. Asimismo, las causales de mayor punibilidad descritas en el art\u00edculo 58 numerales 1, 4, 10\ny 15 ibidem.\n4 Art\u00edculo 350 del C\u00f3digo Penal.\n5 Fl. 65-120.\n\n\nFuente #4\nid: 20240825050857csjsentencia2660114032007pdf_chunk5\ntitle: CSJ - Sentencia 26601(14-03-2007)\nauthor: Corte Suprema de Justicia\nyear: 2007\nkeywords: Penal\ncategory: Jurisprudencia\npage: 7\ncontent: actualmente \u2014Ley 599 de 2000- en los art\u00edculos 411 \u2014Tr\u00e1fico de\nInfluencias-, y 247 (\u00e9n concordancia con el art. 246 ibidem), que\nregula el delito de Estafa con circunstancias de agravaci\u00f3n\npunitiva, para concluir,. conforme lo atribuido al encartado, que la\nilicitud mut\u00f3 en su denominaci\u00f3n t\u00edpica, a esta \u00faltima figura,\nahora asimilada a un punible contra el patrimonio econ\u00f3mico, de\n%r'\u00bf\u00bf*& de %>4\u00fa\u00a1m\u00e1\"\u00bf&\n| P\u00e1gina 8 de 40\n] \u201cS e Segunda instancia N* 26.601\nS BELISARIO MORENO RE\n%nr\u00bfe gy\u00f3rm\u00bfa\u00bb Je%\u00e1!\u00bfm'c\u00bf\nconformidad con lo que sobre el particular ha dejado sentado la\nSala Penal de la H. Corte Suprema de Justicia.\nConcluye, as\u00ed, la sentencia impugnada, que la conducta\npunible siempre ha estado delimitada como tal, aunque, con la\nexpedici\u00f3n de la Ley 590 de 2000, deriv\u00f3 al tipo subordinado de\nestafa agravada, en tanto:\n\u201c..,el procesado MORENO REY, invoc\u00f3 influencia \u2014real o\nsimulada- dada su condici\u00f3n de funcionario de la Fiscal\u00eda\nGeneral de la Naci\u00f3n, para obtener a cambio la suma de un\nMill\u00f3n de pesos ($ 1.000.000.)...\u201d\nM\u00e1s adelante se anot\u00f3, que si bien, no existe evidencia de\nque el acusado efectivamente interviniese ante la Fiscal\nencargada del asunto, para obtener el favor demandado por\nquienes supuestamente le entregaron el dinero, s\u00ed existen\nelementos de juicio suficientes para advertir su inter\u00e9s en el\nproceso llevado por aquella.\nYa en lo tocante a la dosificaci\u00f3n punitiva, el Tribunal\ncontrast\u00f3 las norma en juego, vale decir, art\u00edculo 147 de la Ley\n\n\nFuente #5\nid: 20240820113454ley1142de2007pdf_chunk40\ntitle: Ley 1142 de 2007\nauthor: Congreso\nyear: 2007\nkeywords: Penal\ncategory: Legal\npage: 20\ncontent: Afecta la vigencia de: [Mostrar]\nArt\u00edculo 52. Incluir un numeral 4 al art\u00edculo 247 de la Ley 599 de 2000, el cual quedar\u00e1 as\u00ed:\n4. La conducta est\u00e9 relacionada con contratos de seguros o con transacciones sobre veh\u00edculos automotores.\nAfecta la vigencia de: [Mostrar]\nArt\u00edculo 53. El art\u00edculo 290 de la Ley 599 de 2000, C\u00f3digo Penal, quedar\u00e1 as\u00ed:\nArt\u00edculo 290. Circunstancia de agravaci\u00f3n punitiva. La pena se aumentar\u00e1 hasta en la mitad para el copart\u00edcipe en la\nrealizaci\u00f3n de cualesquiera de las conductas descritas en los art\u00edculos anteriores que usare el documento, salvo en el\nevento del art\u00edculo 289 de este C\u00f3digo.\nSi la conducta recae sobre documentos relacionados con medios motorizados, la pena se incrementar\u00e1 en las tres\ncuartas partes.\nAfecta la vigencia de: [Mostrar]\nArt\u00edculo 54. El art\u00edculo 291 de la Ley 599 de 2000, C\u00f3digo Penal, quedar\u00e1 as\u00ed:\nArt\u00edculo 291. Uso de documento falso. El que sin haber concurrido a la falsificaci\u00f3n hiciere uso de documento p\u00fablico\nfalso que pueda servir de prueba, incurrir\u00e1 en prisi\u00f3n de cuatro (4) a doce (12) a\u00f1os.\nSi la conducta recae sobre documentos relacionados con medios motorizados, el m\u00ednimo de la pena se incrementar\u00e1 en\nla mitad.\nAfecta la vigencia de: [Mostrar]\nArt\u00edculo 55. El art\u00edculo 366 de la Ley 599 de 2000, C\u00f3digo Penal, quedar\u00e1 as\u00ed:\nhttps://www.suin-juriscol.gov.co/viewDocument.asp?ruta=Leyes/1674732# 21/22\n24/6/24, 16:19 LEY 1142 DE 2007\nArt\u00edculo 366. Fabricaci\u00f3n, tr\u00e1fico y porte de armas y municiones de uso privativo de las fuerzas armadas y explosivos. El\n\n\nFuente #6\nid: 20240814181609ley599de2000pdf_chunk169\ntitle: Ley 599 de 2000\nauthor: Congreso\nyear: 2000\nkeywords: Servicios_Publicos\ncategory: Legal\npage: 96\ncontent: penas aumentadas es el siguiente:> El que obtenga provecho il\u00edcito para s\u00ed o para un tercero, con perjuicio ajeno,\ninduciendo o manteniendo a otro en error por medio de artificios o enga\u00f1os, incurrir\u00e1 en prisi\u00f3n de treinta y dos\n(32) a ciento cuarenta y cuatro (144) meses y multa de sesenta y seis punto sesenta y seis (66.66) a mil\nquinientos (1.500) salarios m\u00ednimos legales mensuales vigentes.\nEn la misma pena incurrir\u00e1 el que en loter\u00eda, rifa o juego, obtenga provecho para s\u00ed o para otros, vali\u00e9ndose de\ncualquier medio fraudulento para asegurar un determinado resultado.\nLa pena ser\u00e1 de prisi\u00f3n de diecis\u00e9is (16) a treinta y seis (36) meses y multa hasta de quince (15) salarios m\u00ednimos\nlegales mensuales vigentes, cuando la cuant\u00eda no exceda de diez (10) salarios m\u00ednimos legales mensuales\nvigentes.\nNotas de Vigencia\nConcordancias\nLegislaci\u00f3n Anterior\nIr al inicio\nART\u00cdCULO 247. CIRCUNSTANCIAS DE AGRAVACI\u00d3N PUNITIVA. <Penas aumentadas por el art\u00edculo 14\nde la Ley 890 de 2004, a partir del 1o. de enero de 2005. El texto con las penas aumentadas es el siguiente:> La\npena prevista en el art\u00edculo anterior ser\u00e1 de sesenta y cuatro (64) a ciento cuarenta y cuatro (144) meses cuando:\n1. El medio fraudulento utilizado tenga relaci\u00f3n con vivienda de inter\u00e9s social.\n2. El provecho il\u00edcito se obtenga por quien sin ser part\u00edcipe de un delito de secuestro o extorsi\u00f3n, con ocasi\u00f3n del\nmismo, induzca o mantenga a otro en error.\n3. Se invoquen influencias reales o simuladas con el pretexto o con el fin de obtener de un servidor p\u00fablico un\nbeneficio en asunto que \u00e9ste se encuentre conociendo o haya de conocer.\n\n\nFuente #7\nid: 20240814181609ley599de2000pdf_chunk168\ntitle: Ley 599 de 2000\nauthor: Congreso\nyear: 2000\nkeywords: Servicios_Publicos\ncategory: Legal\npage: 95\ncontent: 4. Cuando se cometa con fines publicitarios o pol\u00edticos constri\u00f1endo a otro mediante amenazas a hacer,\nsuministrar, tolerar u omitir alguna cosa.\n5. Si el prop\u00f3sito o fin perseguido por el agente es facilitar actos terroristas constri\u00f1endo a otro mediante\namenazas a hacer, suministrar, tolerar u omitir alguna cosa.\n6. Cuando se afecten gravemente los bienes o la actividad profesional o econ\u00f3mica de la v\u00edctima.\n7. Si se comete en persona que sea o haya sido periodista, dirigente comunitario, sindical, pol\u00edtico, \u00e9tnico o\nreligioso, o candidato a cargo de elecci\u00f3n popular, en raz\u00f3n de ello, o que sea o hubiere sido servidor p\u00fablico y\npor raz\u00f3n de sus funciones.\n8. Si se comete utilizando orden de captura o detenci\u00f3n falsificada o simulando tenerla, o simulando investidura\no cargo p\u00fablico o fingiere pertenecer a la fuerza p\u00fablica.\n9. Cuando la conducta se comete total o parcialmente desde un lugar de privaci\u00f3n de la libertad.\n10. Si la conducta se comete parcialmente en el extranjero.\n11. En persona internacionalmente protegida diferente o no en el Derecho Internacional Humanitario y agentes\ndiplom\u00e1ticos, de las se\u00f1aladas en los Tratados y Convenios Internacionales ratificados por Colombia.\nNotas de Vigencia\nConcordancias\nLegislaci\u00f3n Anterior\nCAP\u00cdTULO III.\nDE LA ESTAFA\nIr al inicio\nART\u00cdCULO 246. ESTAFA. <Ver Notas de Vigencia en relaci\u00f3n con el art\u00edculo 33 de la Ley 1474 de 2011>\n<Penas aumentadas por el art\u00edculo 14 de la Ley 890 de 2004, a partir del 1o. de enero de 2005. El texto con las\npenas aumentadas es el siguiente:> El que obtenga provecho il\u00edcito para s\u00ed o para un tercero, con perjuicio ajeno,\n\n\nFuente #8\nid: 20240814181609ley599de2000pdf_chunk205\ntitle: Ley 599 de 2000\nauthor: Congreso\nyear: 2000\nkeywords: Servicios_Publicos\ncategory: Legal\npage: 115\ncontent: Concordancias\nIr al inicio\nART\u00cdCULO 301. AGIOTAJE. <Penas aumentadas por el art\u00edculo 14 de la Ley 890 de 2004, a partir del 1o. de\nenero de 2005. El texto con las penas aumentadas es el siguiente:> El que realice maniobra fraudulenta con el fin\nde procurar alteraci\u00f3n en el precio de los art\u00edculos o productos oficialmente considerados de primera necesidad,\nsalarios, materias primas o cualesquiera bienes muebles o inmuebles o servicios que sean objeto de contrataci\u00f3n\nincurrir\u00e1 en prisi\u00f3n de treinta y dos (32) a ciento cuarenta y cuatro (144) meses y multa de sesenta y seis punto\nsesenta y seis (66.66) a setecientos cincuenta (750) salarios m\u00ednimos legales mensuales vigentes.\nLa pena se aumentar\u00e1 hasta en la mitad, si como consecuencia de las conductas anteriores se produjere alguno de\nlos resultados previstos.\n<Inciso 3o. adicionado por el art\u00edculo 20 de la Ley 1474 de 2011. El nuevo texto es el siguiente:> La pena ser\u00e1\nde cinco (5) a\u00f1os a diez (10) a\u00f1os de prisi\u00f3n y multa de cuarenta (40) a mil (1.000) salarios m\u00ednimos legales\nmensuales vigentes, cuando se trate de medicamento o dispositivo m\u00e9dico.\nNotas de Vigencia\nConcordancias\nLegislaci\u00f3n Anterior\nIr al inicio\nART\u00cdCULO 301-A. CIRCUNSTANCIA DE AGRAVACI\u00d3N PUNITIVA. <Art\u00edculo INEXEQUIBLE>\nNotas de Vigencia\nJurisprudencia Vigencia\nLegislaci\u00f3n Anterior\nIr al inicio\nART\u00cdCULO 302. PANICO ECONOMICO. <Penas aumentadas por el art\u00edculo 14 de la Ley 890 de 2004, a partir\ndel 1o. de enero de 2005. El texto con las penas aumentadas es el siguiente:> El que divulgue al p\u00fablico o\n\n\nFuente #9\nid: 20240813213319ley1453de2011pdf_chunk16\ntitle: Ley 1453 de 2011\nauthor: Presidente de la Rep\u00fablica\nyear: 2011\nkeywords: Extinci\u00f3n_de_Dominio\ncategory: Infralegal\npage: 9\ncontent: Afecta la vigencia de: [Mostrar]\nArt\u00edculo 22. El art\u00edculo 377 B de la Ley 599 de 2000 quedar\u00e1 as\u00ed:\nArt\u00edculo 377B.Circunstancia de agravaci\u00f3n punitiva. Si la nave semisumergible o sumergible es utilizada para\nalmacenar, transportar o vender, sustancia estupefaciente, insumos necesarios para su fabricaci\u00f3n o es usado como\nmedio para la comisi\u00f3n de actos delictivos la pena ser\u00e1 de quince (15) a treinta (30) a\u00f1os y multa de setenta mil\n(70.000) salarios m\u00ednimos legales mensuales vigentes.\nLa pena se aumentar\u00e1 de una tercera parte a la mitad cuando la conducta sea realizada por un Servidor P\u00fablico o\nquien haya sido miembro de la Fuerza P\u00fablica.\nAfecta la vigencia de: [Mostrar]\nArt\u00edculo 23. Modif\u00edquese el inciso 2\u201d del art\u00edculo 263 de la Ley 599 de 2000, el cual quedar\u00e1 as\u00ed:\nhttps: //www.suin-juriscol.gov.co/viewDocument.asp?ruta=Leyes/1681231 9/46\n5/7/24, 16:23 LEY 1453 DE 2011\nLa pena establecida en el inciso anterior ser\u00e1 de cuatro (4) a ocho (8) a\u00f1os de prisi\u00f3n para el promotor, organizador\no director de la invasi\u00f3n.\nAfecta la vigencia de: [Mostrar]\nArt\u00edculo 24. Eximente de responsabilidad penal. El art\u00edculo 452 de la Ley 599 de 2000 quedar\u00e1 as\u00ed:\nArt\u00edculo 452.Eximente de responsabilidad penal. Cuando el interno fugado se presentare voluntariamente dentro de\nlas treinta y seis (36) horas siguientes a la evasi\u00f3n, la fuga se tendr\u00e1 en cuenta \u00fanicamente para efectos\ndisciplinarios.\nAfecta la vigencia de: [Mostrar]\nArt\u00edculo 25.Detenci\u00f3n domiciliaria para favorecer la reintegraci\u00f3n del condenado. El art\u00edculo 64 de la Ley 599 de\n\n\nFuente #10\nid: 20240814181609ley599de2000pdf_chunk200\ntitle: Ley 599 de 2000\nauthor: Congreso\nyear: 2000\nkeywords: Servicios_Publicos\ncategory: Legal\npage: 112\ncontent: de la Ley 890 de 2004, a partir del 1o. de enero de 2005. El texto con las penas aumentadas es el siguiente:> El\nque para obtener documento p\u00fablico que pueda servir de prueba, induzca en error a un servidor p\u00fablico, en\nejercicio de sus funciones, haci\u00e9ndole consignar una manifestaci\u00f3n falsa o callar total o parcialmente la verdad,\nincurrir\u00e1 en prisi\u00f3n de cuarenta y ocho (48) a ciento ocho (108) meses.\nNotas de Vigencia\nConcordancias\nLegislaci\u00f3n Anterior\nIr al inicio\nART\u00cdCULO 289. FALSEDAD EN DOCUMENTO PRIVADO. <Penas aumentadas por el art\u00edculo 14 de la Ley\n890 de 2004, a partir del 1o. de enero de 2005. El texto con las penas aumentadas es el siguiente:> El que\nfalsifique documento privado que pueda servir de prueba, incurrir\u00e1, si lo usa, en prisi\u00f3n de diecis\u00e9is (16) a ciento\nocho (108) meses.\nNotas de Vigencia\nJurisprudencia Vigencia\nConcordancias\nLegislaci\u00f3n Anterior\nIr al inicio\nART\u00cdCULO 290. CIRCUNSTANCIA DE AGRAVACI\u00d3N PUNITIVA. <Art\u00edculo modificado por el art\u00edculo 53\nde la Ley 1142 de 2007. El nuevo texto es el siguiente:> La pena se aumentar\u00e1 hasta en la mitad para el\ncopart\u00edcipe en la realizaci\u00f3n de cualesquiera de las conductas descritas en los art\u00edculos anteriores que usare el\ndocumento, salvo en el evento del art\u00edculo 289 de este C\u00f3digo.\nSi la conducta recae sobre documentos relacionados con medios motorizados, la pena se incrementar\u00e1 en las tres\ncuartas partes.\nNotas de Vigencia\nJurisprudencia Vigencia\nLegislaci\u00f3n Anterior\nIr al inicio\nART\u00cdCULO 291. USO DE DOCUMENTO FALSO. <Art\u00edculo modificado por el art\u00edculo 54 de la Ley 1142 de"

In [85]:
query_prompt_message = {"role": "system", "content": response_system_template}
user_prompt_message = {
    "role": "user",
    "content": f"Selecciona las mejores fuentes para responder a: {user_input}",
}
sources_instructions_prompt_message = {
    "role": "system",
    "content": f"Ejectuaste una búsqueda con lo que el usuario te preguntó y encontraste algunas fuentes que pueden servirte, ahora debes escoger las mejores. Sigue las siguientes instrucciones:\n1. Revisa todos los extractos obtenidos con tu búsqueda y escoge entre 3 y 7 fuentes que te sirvan para responder al usuario.\n2. Enlista las fuentes que escogiste e identifícalas por su ID. Ordénalas por importancia, posicionando de primero la que consideres más relevante.\n3. Al lado de cada fuente que seleccionaste escribe una muy breve explicación argumentando porqué la escogiste y porqué ayuda a responder mejor la solicitud del usuario.\n4. Toda la información debe provenir de lo que dicen las fuentes, no inventes ni des información adicional que no estén presentes en las fuentes.",
}
sources_prompt_message = {
    "role": "assistant",
    "content": f"Fuentes econtradas:\n{good_sources}",
}

messages = [
    query_prompt_message,
    # *query_few_shots,
    user_prompt_message,
    sources_instructions_prompt_message,
    sources_prompt_message,
]

print(json.dumps(messages, indent=4))

[
    {
        "role": "system",
        "content": "Eres Ariel, un asistente para la investigaci\u00f3n legal en jurisprudencia colombiana. Sigue estas instrucciones al responder:\n\nConsulta de Fuentes:\n\nUtiliza la herramienta \"search_legal_info\" para buscar informaci\u00f3n relevante.\nCrea consultas espec\u00edficas que incluyan toda la informaci\u00f3n necesaria.\nSi es necesario, genera m\u00faltiples consultas para obtener diferentes tipos de documentos.\nRedacci\u00f3n de Respuestas:\n\nS\u00e9 detallado y preciso, utilizando exclusivamente la informaci\u00f3n de las fuentes proporcionadas.\nNo incluyas informaci\u00f3n que no haya sido extra\u00edda directamente de las fuentes.\nCitaci\u00f3n de Fuentes:\n\nCita el mayor n\u00famero de fuentes aplicables.\nUtiliza corchetes para referenciar la fuente con el ID del fragmento sin modificarlo, por ejemplo: [12345_id_de_ejemplo].\nNo combines fuentes; lista cada fuente por separado.\nFormato de Respuesta:\n\nEscribe tus respu

In [87]:
response = generate_mini_completion()

💰 Conversation price: $17.196015 COP (In: 24105, Out: 964, Total token: 25069)
   This message: $4.1930700000000005 COP (In: 5486, Out: 333, Total token: 5819) 


In [89]:
print(response)
print("\n\n")
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-A7Pg09f0LPqbUJgTBqXxJmFR9etHU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. **Fuente #6 - Ley 599 de 2000 [20240814181609ley599de2000pdf_chunk169]**  \n   Esta fuente es crucial porque detalla las circunstancias de agravación punitiva específicas para el delito de estafa según el Código Penal colombiano.\n\n2. **Fuente #7 - Ley 599 de 2000 [20240814181609ley599de2000pdf_chunk168]**  \n   Complementa la fuente anterior al proporcionar más detalles sobre las circunstancias de agravación punitiva, incluyendo situaciones específicas que agravan el delito de estafa.\n\n3. **Fuente #1 - CSJ - SP23413(15-06-2005) [20240824144301csjsp2341315062005pdf_chunk1]**  \n   Proporciona un análisis jurisprudencial sobre las circunstancias agravantes de la estafa, mencionando específicamente el artículo 247 del Código Penal.\n\n4. **Fuente #2 - CSJ - SP17283(19-03-2002) [20240823230318csjsp1728319032002pdf_chunk18]**